In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 7.7MB/s 
     |████████████████████████████████| 890kB 36.5MB/s 
     |████████████████████████████████| 3.2MB 52.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=9df9208021db1dc3c1ff0706fd56dc4aa88c95127b94c6bb2e442b7e5e604589
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers import BertTokenizer, BertForNextSentencePrediction

# Next Sentence Prediction func

- 사용 가능한 [모델 목록](https://huggingface.co/transformers/pretrained_models.html) 


요청에 따라 모델을 인자로 넘겨주게끔 함수를 짰습니다. 하지만 이 함수를 그대로 사용할 경우, 하나의 문장은 괜찮지만 반복문을 사용할 때 매번 tokenizer, model을 다운로드 받으므로 모델과 토크나이저 선언 부분은 밖으로 빼는 것을 권장합니다!

In [ ]:
def sen_probability(sentence1, sentence2, model_name = "bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForNextSentencePrediction.from_pretrained(model_name)
    encoding = tokenizer(sentence1, sentence2, return_tensors='pt')

    with torch.no_grad():
        outputs = model(**encoding, labels=torch.LongTensor([1]))
        logits = outputs.logits
        prob = logits.softmax(dim=1)
        relat_prob = round(prob[0][0].item(), 4)
        neg_relat_prob = round(prob[0][1].item(), 4)
        pred = "높음" if torch.argmax(prob) == 0 else "낮음"

    return {
        'sentence1': sentence1,
        'sentence2' : sentence2,
        'prediction': pred,
        '다음 문장이 출현할 확률': f"{relat_prob * 100}%",
        '다음 문장이 출현하지 않을 확률': f"{neg_relat_prob * 100}%",
    }

In [ ]:
# test

prompt = "Carl fixed the computer for Margaret. It was because"
next_sentence = "he really liked her."

prompt = "Kristen ran into Howard." 
next_sentence="Howard got hurt."
# Kristen ran into Howard. They talk all the time now.
# Jessica compared grades with Neal. She was doing a little better than him.

sen_probability(prompt, next_sentence,"bert-large-uncased")

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'prediction': '높음',
 'sentence1': 'Kristen ran into Howard.',
 'sentence2': 'Howard got hurt.',
 '다음 문장이 출현하지 않을 확률': '0.0%',
 '다음 문장이 출현할 확률': '100.0%'}

# 작업 수행

여기서부터는 지워도 됩니다. 


In [ ]:
import pandas as pd

In [ ]:
pair = pd.read_csv("/content/nsp_random pair_2.csv")

In [ ]:
pair.head()

,no,code1,sentence1,code2,sentence2
0,1,y,She's the shit.,new,The shelter smells of bodies and bad habits.
1,2,y,You don't understand.,new,All of us are experts.
2,3,y,Ours was the prolonged kind.,new,We're staying focused on the big picture.
3,4,y,Stranger things have happened.,new,Players and teams think they can break them.
4,5,y,Women's Conference begins 9 a.m.,new,And that's why we brought a lawsuit.


In [ ]:
pair.isnull().sum()

no           0
code1        0
sentence1    0
code2        0
sentence2    0
dtype: int64

In [ ]:
prompt=pair['sentence1']
respones=pair['sentence2']

In [ ]:
prompt

0                        She's the shit.
1                  You don't understand.
2           Ours was the prolonged kind.
3         Stranger things have happened.
4       Women's Conference begins 9 a.m.
                     ...                
995                   Give me the match.
996               But there was silence.
997      But someone's paying the bills.
998                     And it was good.
999    But sometimes their systems fail.
Name: sentence1, Length: 1000, dtype: object

In [ ]:
# prompt = pair[0].fillna('missing')
# respones = pair[1].fillna('missing')

In [ ]:
def sen_probability(sentence1, sentence2):
    # if sentence2=="missing": 
    #     print(sentence1, sentence2)
    try:
        encoding = tokenizer(sentence1, sentence2, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**encoding, labels=torch.LongTensor([1]))
            logits = outputs.logits
            prob = logits.softmax(dim=1)
            relat_prob = round(prob[0][0].item()*100, 4)
            neg_relat_prob = round(prob[0][1].item()*100, 4)
            pred = "높음" if torch.argmax(prob) == 0 else "낮음"
        return  relat_prob
    except: 
        print(sentence2)

In [ ]:
model_name = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForNextSentencePrediction.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# bert-base-uncased

prob_bert_base = []
for p,r in zip(prompt, respones):
    prob = sen_probability(p,r)
    prob_bert_base.append(prob)

In [ ]:
# model_name = "bert-large-uncased"

# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForNextSentencePrediction.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# # bert-large-uncased


# prob_bert_large = []
# for p,r in zip(prompt, respones):
#     prob = sen_probability(p,r)
#     prob_bert_large.append(prob)


In [ ]:
prob_bert_base = pd.DataFrame(prob_bert_base)
prob_bert_base
# prob_bert_large = pd.DataFrame(prob_bert_large)


# pair.rename(columns={0:'PROMPT',
#                    1:'RESPONSE'
#                    }, inplace=True)

,0
0,99.8326
1,99.7998
2,0.2313
3,99.9928
4,0.0028
...,...
995,0.1276
996,0.0008
997,0.0065
998,0.0280


In [ ]:
pair["probability"] = prob_bert_base
# pair["bert-large"] = prob_bert_large
pair.head()

,no,code1,sentence1,code2,sentence2,probability
0,1,y,She's the shit.,new,The shelter smells of bodies and bad habits.,99.8326
1,2,y,You don't understand.,new,All of us are experts.,99.7998
2,3,y,Ours was the prolonged kind.,new,We're staying focused on the big picture.,0.2313
3,4,y,Stranger things have happened.,new,Players and teams think they can break them.,99.9928
4,5,y,Women's Conference begins 9 a.m.,new,And that's why we brought a lawsuit.,0.0028


In [ ]:
pair.to_csv("nsp_random_pair_prob.csv")